In [34]:
%pip install matplotlib numpy imageio IPython


Note: you may need to restart the kernel to use updated packages.


In [1]:
from pygltflib import GLTF2, Scene, BufferFormat
from pygltflib.validator import validate, summary
import pathlib
import struct

import numpy
from pygltflib import GLTF2

In [46]:
glb_filename = "./asset/helmet.glb"
gltf = GLTF2().load(glb_filename)
validate(gltf)
current_scene = gltf.scenes[gltf.scene]
current_scene.nodes

/Users/uito/anaconda3/lib/python3.11/site-packages/pygltflib/validator.py:121: UserWarning: pygltf.utils.validator is a provisional function and may not exist in future versions.
  warnings.warn("pygltf.utils.validator is a provisional function and may not exist in future versions.")


[0]

In [24]:
def accessor_type_size(accessor_type):
    """Return the size based on the accessor type."""
    sizes = {
        "SCALAR": 1,
        "VEC2": 2,
        "VEC3": 3,
        "VEC4": 4,
        "MAT2": 4,
        "MAT3": 9,
        "MAT4": 16
    }
    return sizes.get(accessor_type, 1)

def accessor_component_type_size(component_type):
    """Return the size in bytes based on the component type."""
    sizes = {
        5120: 1,  # BYTE
        5121: 1,  # UNSIGNED_BYTE
        5122: 2,  # SHORT
        5123: 2,  # UNSIGNED_SHORT
        5125: 4,  # UNSIGNED_INT
        5126: 4   # FLOAT
    }
    return sizes.get(component_type, 1)

In [30]:
import struct

def decode_binary_data(data, accessor_type, component_type):
    format_char = {
        5120: 'b',  # BYTE
        5121: 'B',  # UNSIGNED_BYTE
        5122: 'h',  # SHORT
        5123: 'H',  # UNSIGNED_SHORT
        5125: 'I',  # UNSIGNED_INT
        5126: 'f'   # FLOAT
    }

    # Number of components per element (e.g., 3 for VEC3)
    num_components = accessor_type_size(accessor_type)

    # Format string for a single element
    element_format = format_char[component_type] * num_components

    # Size in bytes of a single element
    element_size = struct.calcsize(element_format)

    # Decode data
    decoded_data = []
    for i in range(0, len(data), element_size):
        decoded_data.append(struct.unpack(element_format, data[i:i+element_size]))

    return decoded_data

In [50]:
import matplotlib.pyplot as plt

def visualize_vectors(decoded_vectors, accessor_type, elev=None, azim=None):
    if accessor_type == "VEC2":
        x_vals = [vec[0] for vec in decoded_vectors]
        y_vals = [vec[1] for vec in decoded_vectors]
        plt.scatter(x_vals, y_vals)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('VEC2 Visualization')
        plt.show()

    elif accessor_type == "VEC3":
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x_vals = [vec[0] for vec in decoded_vectors]
        y_vals = [vec[1] for vec in decoded_vectors]
        z_vals = [vec[2] for vec in decoded_vectors]
        ax.scatter(x_vals, y_vals, z_vals,s=0.001)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        if elev and azim:
            ax.view_init(elev=elev, azim=azim)
        plt.title('VEC3 Visualization')
        plt.show()


In [91]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from IPython.display import display, Image
import os

def generate_and_display_gif(vector3_tuples):
    # Convert list of tuples to numpy array
    data_np = np.array(vector3_tuples)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data_np[:, 0], data_np[:, 1], data_np[:, 2])

    # Settings for the plot
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    images = []

    degree = 5

    os.makedirs('./temp', exist_ok=True)

    # Rotate the axes and update the plot to create frames for the GIF
    for angle in range(0, 360, degree):  # Rotating by 5 degrees for each frame
        ax.view_init(30, angle)
        plt.draw()
        filename = f'./temp/tmp_{angle}.png'
        plt.savefig(filename)
        images.append(imageio.imread(filename))

    # Create a GIF
    gif_filename = 'rotating_data.gif'
    imageio.mimsave(gif_filename, images, duration=0.1)  # duration is the time for each frame in seconds

    # Cleanup the individual frames
    for angle in range(0, 360, degree):
        filename = f'./temp/tmp_{angle}.png'
        if os.path.exists(filename):
            os.remove(filename)

    # Create a GIF
    gif_filename = 'rotating_data.gif'
    imageio.mimsave(gif_filename, images, duration=0.1)  # duration is the time for each frame in seconds

    # Display the GIF in the notebook
    with open(gif_filename, 'rb') as f:
        display(Image(data=f.read(), format='png'))


In [94]:
import base64

In [93]:
for mesh in gltf.meshes:
    # get the vertices for each primitive in the mesh (in this example there is only one)
    for primitive in mesh.primitives:
        num_vertices = gltf.accessors[primitive.attributes.POSITION].count
        vertex_data = bytearray()
        attributes = {
            "POSITION": primitive.attributes.POSITION,
            "NORMAL": primitive.attributes.NORMAL,
            "TEXCOORD_0": primitive.attributes.TEXCOORD_0,
            "JOINTS_0": primitive.attributes.JOINTS_0,
            "WEIGHTS_0": primitive.attributes.WEIGHTS_0,
        }
        attributes = {k: v for k, v in attributes.items() if v is not None}
        for vertex_index in range(num_vertices):

            for attribute_key, accessor_index in attributes.items():
                accessor_index = attributes[attribute_key]
                accessor = gltf.accessors[accessor_index]
                buffer_view = gltf.bufferViews[accessor.bufferView]
                buffer = gltf.buffers[buffer_view.buffer]
                type_size = accessor_type_size(accessor.type)
                component_size = accessor_component_type_size(accessor.componentType)
                data_start = (
                    buffer_view.byteOffset
                    + accessor.byteOffset
                    + vertex_index * type_size * component_size
                )
                data_end = data_start + type_size * component_size
                data = gltf.get_data_from_buffer_uri(buffer.uri)
                data = data[data_start:data_end]
                vertex_data.extend(data)

        new_buffer = {
            "uri": "data:application/octet-stream;base64," + base64.b64encode(vertex_data).decode("utf-8"),
            "byteLength": len(vertex_data)
        }
        gltf.buffers.append(new_buffer)
        new_buffer_index = len(gltf.buffers) - 1

        current_offset = 0

        # For each primitive and attribute, update the accessor to point to the new buffer
        # for attribute_key, accessor_index in attributes.items():
        #     accessor = gltf.accessors[accessor_index]
        #     accessor.bufferView = new_buffer_index
        #     accessor.byteOffset = current_offset

        #     # Calculate the next offset
        #     attribute_size = accessor_type_size(accessor.type) * accessor_component_type_size(accessor.componentType)
        #     current_offset += attribute_size * accessor.count

        # single_vertex_size = sum(
        #     accessor_type_size(gltf.accessors[accessor_index].type)
        #     * accessor_component_type_size(gltf.accessors[accessor_index].componentType)
        #     for accessor_index in attributes.values()
        # )
        # for attribute_key in attributes:
        #     if attribute_key != "POSITION":
        #         continue
        #     attribute_data = bytearray()
        #     accessor_index = attributes[attribute_key]
        #     accessor = gltf.accessors[accessor_index]
        #     attribute_size = accessor_type_size(
        #         accessor.type
        #     ) * accessor_component_type_size(accessor.componentType)
        #     for i in range(0, len(vertex_data), single_vertex_size):
        #         attribute_data.extend(vertex_data[i : i + attribute_size])
        #     decoded_vectors = decode_binary_data(
        #         attribute_data, accessor.type, accessor.componentType
        #     )

NameError: name 'base64' is not defined

In [ ]:
import matplotlib.pyplot as plt

def visualize_vectors(decoded_vectors, accessor_type):
    if accessor_type == "VEC2":
        x_vals = [vec[0] for vec in decoded_vectors]
        y_vals = [vec[1] for vec in decoded_vectors]
        plt.scatter(x_vals, y_vals)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('VEC2 Visualization')
        plt.show()

    elif accessor_type == "VEC3":
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x_vals = [vec[0] for vec in decoded_vectors]
        y_vals = [vec[1] for vec in decoded_vectors]
        z_vals = [vec[2] for vec in decoded_vectors]
        ax.scatter(x_vals, y_vals, z_vals)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.title('VEC3 Visualization')
        plt.show()

# Call the function
data = buffer.data[data_start:data_end]
decoded_vectors = decode_binary_data(data, accessor.type, accessor.componentType)
visualize_vectors(decoded_vectors, accessor.type)
